<a href="https://colab.research.google.com/github/d-sanjukta/Lyrics-Generator/blob/main/3_Artist_specific_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## <b>3. <u>FINE TUNING TRAINED LLM (ARTIST SPECIFIC TRAINNIG)</b></u>

* We will fine-tune the genre-based language models on five selected artists for each specific genre.
* This process aims to capture the unique style and characteristics of each artist, enabling the generation of content that reflects their artistic expression and creativity.


______________________________________________________________________________________________________________

In [ ]:
# !pip install transformers
# !pip install datasets
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel,pipeline
import re

In [ ]:
# Mounting Google Drive
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


### FUNCTIONS


In [ ]:
## FUNCTION TO PRE_PROCESS THE DATA :

def pre_processing_lyrics(dataset):
  '''
  INPUT : Dataset having different features like : genre , artist , title , view , lyrics etc
  Here, we are interested only in lyrics.
  Output: Processed list of Songs/ lyrics
  '''
  # extracting lyrics from dataset
  try :
    songs  = list(dataset['lyrics'])
  except:
    songs  = list(dataset['Lyric'])


  # REMOVE ANY WORDS CONTAINING NUMERICAL VALUE and SPECIAL CHARACTERS.
  final_lyrics = []
  pattern = re.compile(r'\b[^\W\d_]+\b')  # Regex pattern to match words without numerical characters

  for lyric in songs:
        splits = str(lyric).split(',')
        cleaned_splits = []

        for split in splits:
            cleaned_split = re.sub(r'[^\w\s\']', '', split)
            filtered_split = ' '.join(word for word in cleaned_split.split() if pattern.match(word))
            cleaned_splits.append(filtered_split)

        final_lyrics.append(','.join(cleaned_splits))


  # Applygin 'next line' token at every capital Letter in lyrics and 'end of sequence' token.
  final_lyrics  = [' '.join(['\n' + word if (word[0]).isupper() else word for word in songs.replace('  ', '').split(' ')] + [' </s>']) for songs in final_lyrics]
  print (f"Number of Total Processed Lyrics : {len(final_lyrics)}")

  return final_lyrics

##############################################################################################################################################################################

# Function to make chunks of equal number of words: default = 128 words/tokens

def create_chunks(tokens , window = 128, stride = 20):
  chunks = [] # list to contains chunks of 128 words.
  start = 0
  end = window

  while end < len(tokens):
    chunks.append(tokens[start:end])
    start +=stride
    end +=stride

  return chunks

# Define the  function to take inputs and generate labels and masks.
def create_data(inputs):
    input_ids = inputs[:-1]
    target_ids = inputs[1:]

    return {'input_ids': input_ids, 'attention_mask': tf.ones_like(input_ids)}, target_ids

##################################################################################################################################################################

## FUNCTION TO PREPARE THE 'TENSORFLOW' DATASET:

def prepare_dataset(lyrics, model = 'gpt2',n_words = 128  ):
  '''
  Input : takes in the list of lyrics & the pre-trained tokenizer.
  Output: return the tensorflow datasets.
  '''

  # Split the lyrics into train and validation sets
  val_split = 0.2  # 20% of the data will be used for validation

  split_index = int(len(lyrics) * (1 - val_split))
  train_lyrics = lyrics[:split_index]
  val_lyrics = lyrics[split_index:]

  # Load the pre-trained GPT-2 tokenizer
  tokenizer = GPT2Tokenizer.from_pretrained(model)

  # Tokenize the train and validation lyrics
  encoded_train_lyrics = np.concatenate([tokenizer.encode(l) for l in train_lyrics if tokenizer.encode(l)])
  encoded_val_lyrics = np.concatenate([tokenizer.encode(l) for l in val_lyrics if tokenizer.encode(l)])

  # Creating chunks using the above defined function
  encoded_train_lyrics = create_chunks(encoded_train_lyrics)
  encoded_val_lyrics = create_chunks(encoded_val_lyrics)

  # Restricting the training and validation set to 5000 and 500 respetively. (due to system and time constraints)
  if len(encoded_train_lyrics) > 10000:
    encoded_train_lyrics = encoded_train_lyrics[:10000]
  if len(encoded_val_lyrics) > 500:
    encoded_val_lyrics = encoded_val_lyrics[:500]

  print (f'''
  Size of Training and Validation set:
  Training Size   : {len(encoded_train_lyrics)}
  Validation Size : {len(encoded_val_lyrics)}
  ''')

  # Prepare the tensorflow datasets
  train_dataset = tf.data.Dataset.from_tensor_slices(encoded_train_lyrics)
  val_dataset = tf.data.Dataset.from_tensor_slices(encoded_val_lyrics)

  train_dataset = train_dataset.map(create_data).shuffle(5000).batch(8)
  val_dataset = val_dataset.map(create_data).batch(8)

  return train_dataset, val_dataset

##############################################################################################################################################################################

## FUNCTION TO TRAIN THE MODEL:

def training_model(train_data, validation_data,path,
                   model = 'gpt2',
                   learning_rate = 1e-7,
                   n_epochs = 3):

  # Load the pre-trained GPT-2 small model
  model = TFGPT2LMHeadModel.from_pretrained(model)

  # Loading Weights from drive:
  model.load_weights(path)

  ## Seting up the mmodel:

  # Set up the optimizer and loss function
  optimizer = tf.keras.optimizers.Adam(learning_rate= learning_rate)
  loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

  # Compile the model
  model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'], jit_compile = True)

  # Training Model:

  history  = model.fit(train_data, epochs = n_epochs, validation_data = validation_data)

  return history, model

### TRAINING BEGINS HERE

* We will fine tune all artist specific models for a particular Genre at once.

In [ ]:
# Based on the EDA, the top popular artists from each genre:
genre_artist = {
    'Country' : ['Luke_Combs', 'Johnny_Cash', 'John_Denver', 'Dolly_Parton', 'Morgan_Wallen'],
    'RB'      : ['The_Weekend','Chris_Brown','Dua_Lipa','Ed_Sheeran','Justin_Bieber'],
    'Rock'    : ['Queen', 'The_Beatles', 'Pink_Floyd', 'Maroon5', 'Cold_Play'],
    'Misc'    : ['Scott_Cawthon','Emily_Dickinson', 'Robert_Burns'],
    'Pop'     : ['Taylor_Swift',  'Ariana_Grande', 'Rihanna', 'Ed_Sheeran', 'Lana_Del_Rey'],
    'Rap'     : ['Drake', 'Eminem', 'Kanye_West', 'Kendrick_Lamar', 'Nicki_Minaj']}

#### ARTISTS GENRE : COUNTRY

In [ ]:
genre = 'Country'
artists = genre_artist[genre]

for artist in artists:
  print (f'''
  FINE-TUNING '{genre}' PRE_TRAINED MODEL FOR ARTIST '{artist}'
  ''')

  ## Getting the dataset:
  try:
    # Trying for pickle dataset
    try:
      with open(f'/content/drive/MyDrive/UNIV.AI/AI-3 Language Models/Project/Project Landing /Datasets/Artist_Dataset/{genre}/{artist}.pickle', 'rb') as f:
        df = pickle.load(f)
    # Trying for pandas dataset
    except:
        df = pd.read_csv(f'/content/drive/MyDrive/UNIV.AI/AI-3 Language Models/Project/Project Landing /Datasets/Artist_Dataset/{genre}/{artist}.csv')
  except:
    df = None
    print (f'File not found for artist : {artist}')
    break
  ###############################################################################################################################################################

  # Data Pre-Processing
  songs  = pre_processing_lyrics(df)

  # Preparing Tensorflow Dataset
  train_data , val_data = prepare_dataset(songs)

  # Training Model:
  path = f'/content/drive/MyDrive/UNIV.AI/AI-3 Language Models/Project/Project Landing /Saved Models/Genre Models/{genre}_model_weights.h5'

  _, model = training_model(train_data , val_data , path)

  ################################################################################################################################################################

  # saving model as .h5 file

  model.save_weights(f'/content/drive/MyDrive/UNIV.AI/AI-3 Language Models/Project/Project Landing /Saved Models/Artist Models/{genre}_models/{artist}_weights.h5')

  print ("############################################################################################################################################################################################")



  FINE-TUNING 'Country' PRE_TRAINED MODEL FOR ARTIST 'Luke_Combs' 
  
Number of Total Processed Lyrics : 67

  Size of Training and Validation set: 
  Training Size   : 1155
  Validation Size : 286
  


All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


Epoch 1/3
145/145 [==============================] - 151s 656ms/step - loss: 2.8232 - accuracy: 0.4604 - val_loss: 2.9418 - val_accuracy: 0.4524
Epoch 2/3
145/145 [==============================] - 40s 273ms/step - loss: 2.8153 - accuracy: 0.4616 - val_loss: 2.9395 - val_accuracy: 0.4530
Epoch 3/3
145/145 [==============================] - 44s 304ms/step - loss: 2.8080 - accuracy: 0.4619 - val_loss: 2.9375 - val_accuracy: 0.4534
############################################################################################################################################################################################

  FINE-TUNING 'Country' PRE_TRAINED MODEL FOR ARTIST 'Johnny_Cash' 
  
Number of Total Processed Lyrics : 797


Token indices sequence length is longer than the specified maximum sequence length for this model (1267 > 1024). Running this sequence through the model will result in indexing errors



  Size of Training and Validation set: 
  Training Size   : 5000
  Validation Size : 500
  


...


  Size of Training and Validation set: 
  Training Size   : 5000
  Validation Size : 500
  


All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


Epoch 1/3
625/625 [==============================] - 247s 309ms/step - loss: 2.4416 - accuracy: 0.5100 - val_loss: 2.5068 - val_accuracy: 0.4991
Epoch 2/3
625/625 [==============================] - 184s 295ms/step - loss: 2.4328 - accuracy: 0.5111 - val_loss: 2.5062 - val_accuracy: 0.4991
Epoch 3/3
625/625 [==============================] - 179s 286ms/step - loss: 2.4259 - accuracy: 0.5117 - val_loss: 2.5058 - val_accuracy: 0.4991
############################################################################################################################################################################################

  FINE-TUNING 'Country' PRE_TRAINED MODEL FOR ARTIST 'Morgan_Wallen' 
  
Number of Total Processed Lyrics : 84

  Size of Training and Validation set: 
  Training Size   : 1676
  Validation Size : 290
  


All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


Epoch 1/3
210/210 [==============================] - 175s 557ms/step - loss: 2.8984 - accuracy: 0.4575 - val_loss: 2.7038 - val_accuracy: 0.4852
Epoch 2/3
210/210 [==============================] - 64s 306ms/step - loss: 2.8875 - accuracy: 0.4581 - val_loss: 2.6999 - val_accuracy: 0.4856
Epoch 3/3
210/210 [==============================] - 62s 296ms/step - loss: 2.8791 - accuracy: 0.4596 - val_loss: 2.6967 - val_accuracy: 0.4862
############################################################################################################################################################################################


#### ARTISTS GENRE : RHYTHM & BLUES

In [ ]:
genre = 'RB'
artists = genre_artist[genre]

for artist in artists:
  print (f'''
  FINE-TUNING '{genre}' PRE_TRAINED MODEL FOR ARTIST '{artist}'
  ''')

  ## Getting the dataset:
  try:
    # Trying for pickle dataset
    try:
      with open(f'/content/drive/MyDrive/UNIV.AI/AI-3 Language Models/Project/Project Landing /Datasets/Artist_Dataset/{genre}/{artist}.pickle', 'rb') as f:
        df = pickle.load(f)
    # Trying for pandas dataset
    except:
        df = pd.read_csv(f'/content/drive/MyDrive/UNIV.AI/AI-3 Language Models/Project/Project Landing /Datasets/Artist_Dataset/{genre}/{artist}.csv')
  except:
    df = None
    print (f'File not found for artist : {artist}')
    break
  ###############################################################################################################################################################

  # Data Pre-Processing
  songs  = pre_processing_lyrics(df)

  # Preparing Tensorflow Dataset
  train_data , val_data = prepare_dataset(songs)

  # Training Model:
  path = f'/content/drive/MyDrive/UNIV.AI/AI-3 Language Models/Project/Project Landing /Saved Models/Genre Models/{genre}_model_weights.h5'

  _, model = training_model(train_data , val_data , path)

  ################################################################################################################################################################

  # saving model as .h5 file

  model.save_weights(f'/content/drive/MyDrive/UNIV.AI/AI-3 Language Models/Project/Project Landing /Saved Models/Artist Models/{genre}_models/{artist}_weights.h5')

  print ("############################################################################################################################################################################################")



  FINE-TUNING 'RB' PRE_TRAINED MODEL FOR ARTIST 'The_Weekend' 
  
Number of Total Processed Lyrics : 192


Token indices sequence length is longer than the specified maximum sequence length for this model (1254 > 1024). Running this sequence through the model will result in indexing errors



  Size of Training and Validation set: 
  Training Size   : 4435
  Validation Size : 500
  


All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


Epoch 1/3
555/555 [==============================] - 273s 361ms/step - loss: 2.4290 - accuracy: 0.5284 - val_loss: 2.3490 - val_accuracy: 0.5446
Epoch 2/3
555/555 [==============================] - 161s 290ms/step - loss: 2.4178 - accuracy: 0.5294 - val_loss: 2.3424 - val_accuracy: 0.5449
Epoch 3/3
555/555 [==============================] - 157s 283ms/step - loss: 2.4092 - accuracy: 0.5301 - val_loss: 2.3371 - val_accuracy: 0.5451
############################################################################################################################################################################################

  FINE-TUNING 'RB' PRE_TRAINED MODEL FOR ARTIST 'Chris_Brown' 
  


...

Epoch 1/3
625/625 [==============================] - 242s 294ms/step - loss: 4.0866 - accuracy: 0.3038 - val_loss: 3.7701 - val_accuracy: 0.3027
Epoch 2/3
625/625 [==============================] - 173s 276ms/step - loss: 4.0100 - accuracy: 0.3163 - val_loss: 3.7030 - val_accuracy: 0.3097
Epoch 3/3
625/625 [==============================] - 179s 286ms/step - loss: 3.9668 - accuracy: 0.3212 - val_loss: 3.6591 - val_accuracy: 0.3139
############################################################################################################################################################################################

  FINE-TUNING 'RB' PRE_TRAINED MODEL FOR ARTIST 'Justin_Bieber' 
  
Number of Total Processed Lyrics : 348


Token indices sequence length is longer than the specified maximum sequence length for this model (1148 > 1024). Running this sequence through the model will result in indexing errors



  Size of Training and Validation set: 
  Training Size   : 5000
  Validation Size : 500
  


All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


Epoch 1/3
625/625 [==============================] - 252s 310ms/step - loss: 3.5517 - accuracy: 0.3604 - val_loss: 3.2180 - val_accuracy: 0.4141
Epoch 2/3
625/625 [==============================] - 184s 294ms/step - loss: 3.4721 - accuracy: 0.3729 - val_loss: 3.1607 - val_accuracy: 0.4220
Epoch 3/3
625/625 [==============================] - 184s 294ms/step - loss: 3.4261 - accuracy: 0.3781 - val_loss: 3.1212 - val_accuracy: 0.4272
############################################################################################################################################################################################


#### ARTISTS GENRE : ROCK

In [ ]:
genre = 'Rock'
artists = genre_artist[genre]

for artist in artists:
  print (f'''
  FINE-TUNING '{genre}' PRE_TRAINED MODEL FOR ARTIST '{artist}'
  ''')

  ## Getting the dataset:
  try:
    # Trying for pickle dataset
    try:
      with open(f'/content/drive/MyDrive/UNIV.AI/AI-3 Language Models/Project/Project Landing /Datasets/Artist_Dataset/{genre}/{artist}.pickle', 'rb') as f:
        df = pickle.load(f)
    # Trying for pandas dataset
    except:
        df = pd.read_csv(f'/content/drive/MyDrive/UNIV.AI/AI-3 Language Models/Project/Project Landing /Datasets/Artist_Dataset/{genre}/{artist}.csv')
  except:
    df = None
    print (f'File not found for artist : {artist}')
    break
  ###############################################################################################################################################################

  # Data Pre-Processing
  songs  = pre_processing_lyrics(df)

  # Preparing Tensorflow Dataset
  train_data , val_data = prepare_dataset(songs)

  # Training Model:
  path = f'/content/drive/MyDrive/UNIV.AI/AI-3 Language Models/Project/Project Landing /Saved Models/Genre Models/{genre}_model_weights.h5'

  _, model = training_model(train_data , val_data , path)

  ################################################################################################################################################################

  # saving model as .h5 file

  model.save_weights(f'/content/drive/MyDrive/UNIV.AI/AI-3 Language Models/Project/Project Landing /Saved Models/Artist Models/{genre}_models/{artist}_weights.h5')

  print ("############################################################################################################################################################################################")



  FINE-TUNING 'Rock' PRE_TRAINED MODEL FOR ARTIST 'Queen' 
  
Number of Total Processed Lyrics : 475

  Size of Training and Validation set: 
  Training Size   : 5000
  Validation Size : 500
  


All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


Epoch 1/3
625/625 [==============================] - 234s 277ms/step - loss: 2.5345 - accuracy: 0.5138 - val_loss: 2.3193 - val_accuracy: 0.5489
Epoch 2/3
625/625 [==============================] - 161s 257ms/step - loss: 2.5236 - accuracy: 0.5148 - val_loss: 2.3086 - val_accuracy: 0.5499
Epoch 3/3
625/625 [==============================] - 161s 258ms/step - loss: 2.5139 - accuracy: 0.5156 - val_loss: 2.2996 - val_accuracy: 0.5510
############################################################################################################################################################################################

  FINE-TUNING 'Rock' PRE_TRAINED MODEL FOR ARTIST 'The_Beatles' 
  
Number of Total Processed Lyrics : 557


Token indices sequence length is longer than the specified maximum sequence length for this model (1778 > 1024). Running this sequence through the model will result in indexing errors



  Size of Training and Validation set: 
  Training Size   : 5000
  Validation Size : 500
  


...

Epoch 1/3
317/317 [==============================] - 203s 437ms/step - loss: 2.9347 - accuracy: 0.4604 - val_loss: 2.8373 - val_accuracy: 0.4675
Epoch 2/3
317/317 [==============================] - 82s 259ms/step - loss: 2.9232 - accuracy: 0.4617 - val_loss: 2.8286 - val_accuracy: 0.4689
Epoch 3/3
317/317 [==============================] - 85s 266ms/step - loss: 2.9173 - accuracy: 0.4621 - val_loss: 2.8208 - val_accuracy: 0.4699
############################################################################################################################################################################################

  FINE-TUNING 'Rock' PRE_TRAINED MODEL FOR ARTIST 'Maroon5' 
  
Number of Total Processed Lyrics : 197

  Size of Training and Validation set: 
  Training Size   : 3029
  Validation Size : 500
  


All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


Epoch 1/3
379/379 [==============================] - 219s 409ms/step - loss: 3.4969 - accuracy: 0.3611 - val_loss: 3.3603 - val_accuracy: 0.3835
Epoch 2/3
379/379 [==============================] - 98s 258ms/step - loss: 3.4179 - accuracy: 0.3759 - val_loss: 3.2999 - val_accuracy: 0.3932
Epoch 3/3
379/379 [==============================] - 99s 262ms/step - loss: 3.3690 - accuracy: 0.3829 - val_loss: 3.2585 - val_accuracy: 0.3999
############################################################################################################################################################################################

  FINE-TUNING 'Rock' PRE_TRAINED MODEL FOR ARTIST 'Cold_Play' 
  
Number of Total Processed Lyrics : 223

  Size of Training and Validation set: 
  Training Size   : 2719
  Validation Size : 500
  


All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


Epoch 1/3
340/340 [==============================] - 209s 421ms/step - loss: 2.4130 - accuracy: 0.5309 - val_loss: 2.2607 - val_accuracy: 0.5553
Epoch 2/3
340/340 [==============================] - 88s 260ms/step - loss: 2.4048 - accuracy: 0.5318 - val_loss: 2.2551 - val_accuracy: 0.5552
Epoch 3/3
340/340 [==============================] - 88s 258ms/step - loss: 2.3991 - accuracy: 0.5321 - val_loss: 2.2502 - val_accuracy: 0.5558
############################################################################################################################################################################################


#### ARTISTS GENRE : POP

In [ ]:
genre = 'Pop'
artists = genre_artist[genre]

for artist in artists:
  print (f'''
  FINE-TUNING '{genre}' PRE_TRAINED MODEL FOR ARTIST '{artist}'
  ''')

  ## Getting the dataset:
  try:
    # Trying for pickle dataset
    try:
      with open(f'/content/drive/MyDrive/UNIV.ai/Project Landing /Datasets/Artist_Dataset/{genre}/{artist}.pickle', 'rb') as f:
        df = pickle.load(f)
    # Trying for pandas dataset
    except:
        df = pd.read_csv(f'/content/drive/MyDrive/UNIV.ai/Project Landing /Datasets/Artist_Dataset/{genre}/{artist}.csv')
  except:
    df = None
    print (f'File not found for artist : {artist}')
    break
  ###############################################################################################################################################################

  # Data Pre-Processing
  songs  = pre_processing_lyrics(df)

  # Preparing Tensorflow Dataset
  train_data , val_data = prepare_dataset(songs)

  # Training Model:
  path = f'/content/drive/MyDrive/UNIV.ai/Project Landing /Saved Models/Genre Models/{genre}_model_weights.h5'

  _, model = training_model(train_data , val_data , path)

  ################################################################################################################################################################

  # saving model as .h5 file

  model.save_weights(f'/content/drive/MyDrive/UNIV.ai/Project Landing /Saved Models/Artist Models/{genre}_models/{artist}_weights.h5')

  print ("############################################################################################################################################################################################")



  FINE-TUNING 'Pop' PRE_TRAINED MODEL FOR ARTIST 'Taylor_Swift' 
  
Number of Total Processed Lyrics : 479


Token indices sequence length is longer than the specified maximum sequence length for this model (2909 > 1024). Running this sequence through the model will result in indexing errors



  Size of Training and Validation set: 
  Training Size   : 5000
  Validation Size : 500
  


All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


Epoch 1/3
625/625 [==============================] - 230s 280ms/step - loss: 3.8649 - accuracy: 0.3152 - val_loss: 4.1921 - val_accuracy: 0.2750
Epoch 2/3
625/625 [==============================] - 161s 258ms/step - loss: 3.7697 - accuracy: 0.3319 - val_loss: 4.1252 - val_accuracy: 0.2863
Epoch 3/3
625/625 [==============================] - 164s 262ms/step - loss: 3.7183 - accuracy: 0.3389 - val_loss: 4.0852 - val_accuracy: 0.2915
############################################################################################################################################################################################

  FINE-TUNING 'Pop' PRE_TRAINED MODEL FOR ARTIST 'Ariana_Grande' 
  
Number of Total Processed Lyrics : 252


...

Epoch 1/3
625/625 [==============================] - 235s 285ms/step - loss: 2.7193 - accuracy: 0.4864 - val_loss: 2.5592 - val_accuracy: 0.5010
Epoch 2/3
625/625 [==============================] - 165s 264ms/step - loss: 2.7051 - accuracy: 0.4879 - val_loss: 2.5578 - val_accuracy: 0.5016
Epoch 3/3
625/625 [==============================] - 165s 264ms/step - loss: 2.6941 - accuracy: 0.4893 - val_loss: 2.5566 - val_accuracy: 0.5025
############################################################################################################################################################################################

  FINE-TUNING 'Pop' PRE_TRAINED MODEL FOR ARTIST 'Lana_Del_Rey' 
  
Number of Total Processed Lyrics : 436


Token indices sequence length is longer than the specified maximum sequence length for this model (1185 > 1024). Running this sequence through the model will result in indexing errors



  Size of Training and Validation set: 
  Training Size   : 5000
  Validation Size : 500
  


All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


Epoch 1/3
625/625 [==============================] - 243s 287ms/step - loss: 2.5003 - accuracy: 0.5132 - val_loss: 2.4905 - val_accuracy: 0.5360
Epoch 2/3
625/625 [==============================] - 166s 265ms/step - loss: 2.4892 - accuracy: 0.5145 - val_loss: 2.4824 - val_accuracy: 0.5371
Epoch 3/3
625/625 [==============================] - 165s 263ms/step - loss: 2.4797 - accuracy: 0.5150 - val_loss: 2.4756 - val_accuracy: 0.5379
############################################################################################################################################################################################


#### ARTISTS GENRE : RAP

In [ ]:
genre = 'Rap'
artists = genre_artist[genre]

for artist in artists:
  print (f'''
  FINE-TUNING '{genre}' PRE_TRAINED MODEL FOR ARTIST '{artist}'
  ''')

  ## Getting the dataset:
  try:
    # Trying for pickle dataset
    try:
      with open(f'/content/drive/MyDrive/UNIV.AI/Project AI 3 NLP/Project Landing /Datasets/Artist_Dataset/{genre}/{artist}.pickle', 'rb') as f:
        df = pickle.load(f)
    # Trying for pandas dataset
    except:
        df = pd.read_csv(f'/content/drive/MyDrive/UNIV.AI/Project AI 3 NLP/Project Landing /Datasets/Artist_Dataset/{genre}/{artist}.csv')
  except:
    df = None
    print (f'File not found for artist : {artist}')
    break
  ###############################################################################################################################################################

  # Data Pre-Processing
  songs  = pre_processing_lyrics(df)

  # Preparing Tensorflow Dataset
  train_data , val_data = prepare_dataset(songs)

  # Training Model:
  path = f'/content/drive/MyDrive/UNIV.AI/Project AI 3 NLP/Project Landing /Saved Models/Genre Models/{genre}_model_weights.h5'

  _, model = training_model(train_data , val_data , path)

  ################################################################################################################################################################

  # saving model as .h5 file

  model.save_weights(f'/content/drive/MyDrive/UNIV.AI/Project AI 3 NLP/Project Landing /Saved Models/Artist Models/{genre}_models/{artist}_weights.h5')

  print ("############################################################################################################################################################################################")



  FINE-TUNING 'Rap' PRE_TRAINED MODEL FOR ARTIST 'Drake' 
  
Number of Total Processed Lyrics : 426


Token indices sequence length is longer than the specified maximum sequence length for this model (1184 > 1024). Running this sequence through the model will result in indexing errors



  Size of Training and Validation set: 
  Training Size   : 10000
  Validation Size : 500
  


All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


Epoch 1/3
1250/1250 [==============================] - 397s 267ms/step - loss: 2.9922 - accuracy: 0.4475 - val_loss: 3.1072 - val_accuracy: 0.4389
Epoch 2/3
1250/1250 [==============================] - 328s 262ms/step - loss: 2.9800 - accuracy: 0.4487 - val_loss: 3.1019 - val_accuracy: 0.4400
Epoch 3/3
1250/1250 [==============================] - 332s 266ms/step - loss: 2.9708 - accuracy: 0.4497 - val_loss: 3.0983 - val_accuracy: 0.4408
############################################################################################################################################################################################

  FINE-TUNING 'Rap' PRE_TRAINED MODEL FOR ARTIST 'Eminem' 
  
Number of Total Processed Lyrics : 496


...

Epoch 1/3
1250/1250 [==============================] - 399s 276ms/step - loss: 3.2313 - accuracy: 0.4263 - val_loss: 3.2893 - val_accuracy: 0.4219
Epoch 2/3
1250/1250 [==============================] - 333s 267ms/step - loss: 3.2189 - accuracy: 0.4277 - val_loss: 3.2844 - val_accuracy: 0.4224
Epoch 3/3
1250/1250 [==============================] - 333s 267ms/step - loss: 3.2099 - accuracy: 0.4283 - val_loss: 3.2805 - val_accuracy: 0.4225
############################################################################################################################################################################################

  FINE-TUNING 'Rap' PRE_TRAINED MODEL FOR ARTIST 'Nicki_Minaj' 
  
Number of Total Processed Lyrics : 323


Token indices sequence length is longer than the specified maximum sequence length for this model (1063 > 1024). Running this sequence through the model will result in indexing errors



  Size of Training and Validation set: 
  Training Size   : 7054
  Validation Size : 500
  


All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


Epoch 1/3
882/882 [==============================] - 339s 321ms/step - loss: 4.3668 - accuracy: 0.2852 - val_loss: 4.2141 - val_accuracy: 0.2973
Epoch 2/3
882/882 [==============================] - 233s 264ms/step - loss: 4.2307 - accuracy: 0.3057 - val_loss: 4.1472 - val_accuracy: 0.3097
Epoch 3/3
882/882 [==============================] - 232s 263ms/step - loss: 4.1651 - accuracy: 0.3160 - val_loss: 4.1134 - val_accuracy: 0.3158
############################################################################################################################################################################################


#### ARTISTS GENRE : MISCELLANEOUS

In [ ]:
genre = 'Misc'
artists = genre_artist[genre]

for artist in artists:
  print (f'''
  FINE-TUNING '{genre}' PRE_TRAINED MODEL FOR ARTIST '{artist}'
  ''')

  ## Getting the dataset:
  try:
    # Trying for pickle dataset
    try:
      with open(f'/content/drive/MyDrive/UNIV.ai/Project Landing /Datasets/Artist_Dataset/{genre}/{artist}.pickle', 'rb') as f:
        df = pickle.load(f)
    # Trying for pandas dataset
    except:
        df = pd.read_csv(f'/content/drive/MyDrive/UNIV.ai/Project Landing /Datasets/Artist_Dataset/{genre}/{artist}.csv')
  except:
    df = None
    print (f'File not found for artist : {artist}')
    break
  ###############################################################################################################################################################

  # Data Pre-Processing
  songs  = pre_processing_lyrics(df)

  # Preparing Tensorflow Dataset
  train_data , val_data = prepare_dataset(songs)

  # Training Model:
  path = f'/content/drive/MyDrive/UNIV.ai/Project Landing /Saved Models/Genre Models/{genre}_model_weights.h5'

  _, model = training_model(train_data , val_data , path)

  ################################################################################################################################################################

  # saving model as .h5 file

  model.save_weights(f'/content/drive/MyDrive/UNIV.ai/Project Landing /Saved Models/Artist Models/{genre}_models/{artist}_weights.h5')

  print ("############################################################################################################################################################################################")


Token indices sequence length is longer than the specified maximum sequence length for this model (1879 > 1024). Running this sequence through the model will result in indexing errors



  FINE-TUNING 'Misc' PRE_TRAINED MODEL FOR ARTIST 'Scott_Cawthon' 
  
Number of Total Processed Lyrics : 87

  Size of Training and Validation set: 
  Training Size   : 2012
  Validation Size : 327
  


All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


Epoch 1/3
252/252 [==============================] - 173s 454ms/step - loss: 3.3668 - accuracy: 0.3814 - val_loss: 3.0136 - val_accuracy: 0.4106
Epoch 2/3
252/252 [==============================] - 69s 272ms/step - loss: 3.3383 - accuracy: 0.3844 - val_loss: 2.9989 - val_accuracy: 0.4128
Epoch 3/3
252/252 [==============================] - 68s 271ms/step - loss: 3.3167 - accuracy: 0.3873 - val_loss: 2.9879 - val_accuracy: 0.4135
############################################################################################################################################################################################

  FINE-TUNING 'Misc' PRE_TRAINED MODEL FOR ARTIST 'Emily_Dickinson' 
  
Number of Total Processed Lyrics : 1167


Token indices sequence length is longer than the specified maximum sequence length for this model (1057 > 1024). Running this sequence through the model will result in indexing errors


...

Epoch 1/3
625/625 [==============================] - 235s 288ms/step - loss: 3.6446 - accuracy: 0.4212 - val_loss: 3.7121 - val_accuracy: 0.3076
Epoch 2/3
625/625 [==============================] - 173s 276ms/step - loss: 3.5988 - accuracy: 0.4282 - val_loss: 3.7239 - val_accuracy: 0.3029
Epoch 3/3
625/625 [==============================] - 168s 268ms/step - loss: 3.5718 - accuracy: 0.4301 - val_loss: 3.7354 - val_accuracy: 0.3003
############################################################################################################################################################################################

  FINE-TUNING 'Misc' PRE_TRAINED MODEL FOR ARTIST 'Robert_Burns' 
  
Number of Total Processed Lyrics : 544


Token indices sequence length is longer than the specified maximum sequence length for this model (1310 > 1024). Running this sequence through the model will result in indexing errors



  Size of Training and Validation set: 
  Training Size   : 5000
  Validation Size : 500
  


All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


Epoch 1/3
625/625 [==============================] - 238s 297ms/step - loss: 4.6275 - accuracy: 0.2618 - val_loss: 4.1463 - val_accuracy: 0.3186
Epoch 2/3
625/625 [==============================] - 172s 276ms/step - loss: 4.5956 - accuracy: 0.2643 - val_loss: 4.1282 - val_accuracy: 0.3206
Epoch 3/3
625/625 [==============================] - 172s 275ms/step - loss: 4.5710 - accuracy: 0.2665 - val_loss: 4.1148 - val_accuracy: 0.3223
############################################################################################################################################################################################
